In [1]:
import os, sys, glob, shutil
import argparse
import json
import yaml
import numpy as np
from pprint import pprint

import torch
import yaml
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from scipy.io import wavfile
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

In [2]:
%cd "D:\Schoolwork\TERM 3\WORK\visual_prosody"

D:\Schoolwork\TERM 3\WORK\visual_prosody


In [3]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [4]:
print(len(val_uids))

2772


In [5]:
from utils.model import get_model, get_vocoder, get_param_num, vocoder_infer
from utils.tools import to_device, log, synth_one_sample, expand, plot_mel
from model import FastSpeech2Loss
from dataset import Dataset
# from utils.auto_tqdm import tqdm
from evaluate import evaluate

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
parser = argparse.ArgumentParser()
parser.add_argument("--restore_step", type=int, default=0)
parser.add_argument(
    "-p",
    "--preprocess_config",
    type=str,
    required=True,
    help="path to preprocess.yaml",
)
parser.add_argument(
    "-m", "--model_config", type=str, required=True, help="path to model.yaml"
)
parser.add_argument(
    "-t", "--train_config", type=str, required=True, help="path to train.yaml"
)

argString = '-p ./config/Ego4D_final_v6/0703b_preprocess.yaml -m ./config/Ego4D_final_v6/0703b_model.yaml -t ./config/Ego4D_final_v6/0703b_train.yaml'
# args = parser.parse_args()
args = parser.parse_args(argString.split())

In [8]:
pprint(args)
# Read Config
preprocess_config = yaml.load(
    open(args.preprocess_config, "r"), Loader=yaml.FullLoader
)
model_config = yaml.load(open(args.model_config, "r"), Loader=yaml.FullLoader)
train_config = yaml.load(open(args.train_config, "r"), Loader=yaml.FullLoader)
configs = (preprocess_config, model_config, train_config)
print("Prepare training ...")

preprocess_config, model_config, train_config = configs

Namespace(restore_step=0, preprocess_config='./config/Ego4D_final_v6/0703b_preprocess.yaml', model_config='./config/Ego4D_final_v6/0703b_model.yaml', train_config='./config/Ego4D_final_v6/0703b_train.yaml')
Prepare training ...


In [9]:
ckpt_path = r'./output/0703b/ckpt/Ego4D_final_v6/1200000.pth.tar'
ckpt = torch.load(ckpt_path)

In [10]:
# Prepare model
model, optimizer = get_model(args, configs, device, train=True)
model.load_state_dict(ckpt["model"], strict=True)
model.to(device)
model = nn.DataParallel(model)
num_param = get_param_num(model)
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)
print("Number of FastSpeech2 Parameters:", num_param)

=> Not using speaker embeddings.
False
None
Number of FastSpeech2 Parameters: 35159745


In [11]:
# Load vocoder
vocoder = get_vocoder(model_config, device)
step = args.restore_step + 1
model.eval()
print()
dataset = Dataset(
    "val.txt", 'val', preprocess_config, train_config, sort=False, drop_last=False
)

Removing weight norm...



In [12]:
batch_size = train_config["optimizer"]["batch_size"]
batch_size = 1

In [13]:
loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=dataset.collate_fn,
    )
# Get loss function
Loss = FastSpeech2Loss(preprocess_config, model_config).to(device)

In [14]:
os.path.join(train_config['path']['result_path'], 'plots')

'./output/0703b/result/Ego4D_final_v6\\plots'

In [15]:
# => batch:
# return (
#     ids,
#     raw_texts,
#     speakers,
#     texts,
#     text_lens,
#     max(text_lens),
#     mels,
#     mel_lens,
#     max(mel_lens),
#     pitches,
#     energies,
#     durations,
#     speaker_embeddings,
# )
# [12] 对应speaker embedding


# => model output / prediction
# return (
#     output,
#     postnet_output,
#     p_predictions,
#     e_predictions,
#     log_d_predictions,
#     d_rounded,
#     src_masks,
#     mel_masks,
#     src_lens,
#     mel_lens,
# )
output_plot_path = os.path.join(train_config['path']['result_path'], 'plot')
output_mel_syn_path = os.path.join(train_config['path']['result_path'], 'mel', 'syn')
output_mel_gt_path = os.path.join(train_config['path']['result_path'], 'mel', 'gt')
output_wav_syn_path = os.path.join(train_config['path']['result_path'], 'wav', 'synthesized')
output_wav_rec_path = os.path.join(train_config['path']['result_path'], 'wav', 'reconstructed')
os.makedirs(output_plot_path, exist_ok=True)
os.makedirs(output_mel_syn_path, exist_ok=True)
os.makedirs(output_mel_gt_path, exist_ok=True)
os.makedirs(output_wav_syn_path, exist_ok=True)
os.makedirs(output_wav_rec_path, exist_ok=True)

In [16]:
for batchs in tqdm(loader):
    for targets in batchs:
        targets = to_device(targets, device)
        with torch.no_grad():
            predictions = model(*(targets[2:]))
        basenames = targets[0]
        for i in range(len(predictions[0])):
            basename = basenames[i]
            src_len = predictions[8][i].item()
            mel_len = predictions[9][i].item()
            mel_prediction = predictions[1][i, :mel_len].detach().transpose(0, 1)
            mel_target = targets[6][i, :mel_len].detach().transpose(0, 1)

            torch.save(mel_prediction.cpu(), os.path.join(output_mel_syn_path, f"{basename}.pt"))
            torch.save(mel_target.cpu(), os.path.join(output_mel_gt_path, f"{basename}.pt"))
            
            
            duration = predictions[5][i, :src_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["pitch"]["feature"] == "phoneme_level":
                pitch = predictions[2][i, :src_len].detach().cpu().numpy()
                pitch = expand(pitch, duration)
            else:
                pitch = predictions[2][i, :mel_len].detach().cpu().numpy()
            if preprocess_config["preprocessing"]["energy"]["feature"] == "phoneme_level":
                energy = predictions[3][i, :src_len].detach().cpu().numpy()
                energy = expand(energy, duration)
            else:
                energy = predictions[3][i, :mel_len].detach().cpu().numpy()

            with open(os.path.join(preprocess_config["path"]["preprocessed_path"], 
                                   "stats.json")) as f:
                stats = json.load(f)
                stats = stats["pitch"] + stats["energy"][:2]
                                       
            fig = plot_mel(
                [
                    (mel_prediction.cpu().numpy(), pitch, energy),
                    (mel_target.cpu().numpy(), pitch, energy),
                ],
                stats,
                ["Synthetized Spectrogram", "Ground-Truth Spectrogram"],
            )
            ### TODO: change to svg
            plt.savefig(os.path.join(output_plot_path, f"{basename}.png"))
            plt.close()

        # from .model import vocoder_infer

        mel_predictions = predictions[1].transpose(1, 2)
        mel_targets = targets[6].transpose(1, 2)
        
        lengths = predictions[9] * preprocess_config["preprocessing"]["stft"]["hop_length"]
        wav_predictions = vocoder_infer(
            mel_predictions, vocoder, model_config, preprocess_config, lengths=lengths
        )
        wav_targets = vocoder_infer(
        mel_targets, vocoder, model_config, preprocess_config, lengths=lengths
    )
    
        sampling_rate = preprocess_config["preprocessing"]["audio"]["sampling_rate"]
        for wav, basename in zip(wav_predictions, basenames):
            wavfile.write(os.path.join(output_wav_syn_path, f"{basename}.wav"), sampling_rate, wav)
        for wav, basename in zip(wav_targets, basenames):
            wavfile.write(os.path.join(output_wav_rec_path, f"{basename}.wav"), sampling_rate, wav)

        
    #     break
    # break

  0%|          | 0/2772 [00:00<?, ?it/s]

# MCD

In [17]:
import os, sys, glob, shutil
import os.path as op
import argparse
import json
import yaml
import numpy as np
import pandas as pd
from pprint import pprint
import os.path as op

import torch
import torch.nn.functional as F
import torchaudio
import argparse
import fnmatch
import logging
import multiprocessing as mp
from typing import Dict, List, Tuple

import librosa
import pysptk
import pyworld as pw
import soundfile as sf
from fastdtw import fastdtw
from scipy import spatial

In [18]:
split_txt_val_path = r'.\preprocessed_data\Ego4D_final_v6\val.txt'
val_uids = []
with open(split_txt_val_path) as file:
    for line in file:
        # print(line.split('|')[0])
        val_uids.append(line.split('|')[0])

In [19]:
len(val_uids)

2772

In [20]:
recon_folder = r'.\output\0703b\result\Ego4D_final_v6\wav\reconstructed'
syn_folder = r'.\output\0703b\result\Ego4D_final_v6\wav\synthesized'
gt_folder = r'.\raw_data\Ego4D_final_v6\val\Ego4D_final_v6'

In [21]:
sr = 22050
melkwargs = {
    "n_fft": int(0.05 * sr), "win_length": int(0.05 * sr),
    "hop_length": int(0.0125 * sr), "f_min": 20,
    "n_mels": 80, "window_fn": torch.hann_window
}
mfcc_fn = torchaudio.transforms.MFCC(
    sr, n_mfcc=13, log_mels=True, melkwargs=melkwargs
)

In [22]:
%%time
uids = []
MCDs_recon = []
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(recon_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_recon.append(result)

  0%|          | 0/2772 [00:00<?, ?it/s]

CPU times: total: 10min 11s
Wall time: 1min 48s


In [23]:
%%time
uids = []
MCDs_gt = []
for uid in tqdm(val_uids):
    uids.append(uid)
    wav_1, sr = torchaudio.load(op.join(gt_folder, f"{uid}.wav"))
    wav_2, sr = torchaudio.load(op.join(syn_folder, f"{uid}.wav"))
    wav_1 = wav_1.squeeze()
    wav_2 = wav_2.squeeze()
    mel_1 = mfcc_fn(wav_1).T.numpy()
    mel_2 = mfcc_fn(wav_2).T.numpy()
    # DTW
    _, path = fastdtw(mel_2, mel_1, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    mel_2 = mel_2[twf[0]]
    mel_1 = mel_1[twf[1]]
    # We sum the squared differences over the first K MFCCs, skipping ct,0
    mel_1 = mel_1[:, 1:]
    mel_2 = mel_2[:, 1:]
    result = (((mel_1 - mel_2) ** 2).sum(axis=1)**0.5).mean()
    MCDs_gt.append(result)

  0%|          | 0/2772 [00:00<?, ?it/s]

CPU times: total: 9min 45s
Wall time: 1min 41s


In [24]:
v6_mcd_df = pd.DataFrame({
    'uid': uids,
    'MCD_recon': MCDs_recon,
    'MCD_gt': MCDs_gt,
})

In [25]:
v6_mcd_df.to_csv(r".\jupyter_walkthrough\metrics\MCD_0703b_1M2.csv")

In [38]:
print(f"MCDs_recon mean on 0703b_1M2: {torch.tensor(MCDs_recon).mean()}")
print(f"MCDs_recon std on 0703b_1M2: {torch.tensor(MCDs_recon).std()}")

print(f"MCDs_gt mean on 0703b_1M2: {torch.tensor(MCDs_gt).mean()}")
print(f"MCDs_gt std on 0703b_1M2: {torch.tensor(MCDs_gt).std()}")

MCDs_recon mean on 0703b_1M2: 13.177790641784668
MCDs_recon std on 0703b_1M2: 2.5031161308288574
MCDs_gt mean on 0703b_1M2: 14.175084114074707
MCDs_gt std on 0703b_1M2: 2.488023281097412


# log f0

In [27]:
# https://github.com/espnet/espnet/blob/3e0dad524d62ccd45e067e9b36049f2214ea972a/egs2/TEMPLATE/asr1/pyscripts/utils/evaluate_f0.py

In [28]:
def world_extract(
    x: np.ndarray,
    fs: int,
    f0min: int = 40,
    f0max: int = 800,
    n_fft: int = 512,
    n_shift: int = 256,
    mcep_dim: int = 25,
    mcep_alpha: float = 0.41,
) -> np.ndarray:
    """Extract World-based acoustic features.

    Args:
        x (ndarray): 1D waveform array.
        fs (int): Minimum f0 value (default=40).
        f0 (int): Maximum f0 value (default=800).
        n_shift (int): Shift length in point (default=256).
        n_fft (int): FFT length in point (default=512).
        n_shift (int): Shift length in point (default=256).
        mcep_dim (int): Dimension of mel-cepstrum (default=25).
        mcep_alpha (float): All pass filter coefficient (default=0.41).

    Returns:
        ndarray: Mel-cepstrum with the size (N, n_fft).
        ndarray: F0 sequence (N,).

    """
    # extract features
    x = x.astype(np.float64)
    f0, time_axis = pw.harvest(
        x,
        fs,
        f0_floor=f0min,
        f0_ceil=f0max,
        frame_period=n_shift / fs * 1000,
    )
    sp = pw.cheaptrick(x, f0, time_axis, fs, fft_size=n_fft)
    if mcep_dim is None or mcep_alpha is None:
        mcep_dim, mcep_alpha = _get_best_mcep_params(fs)
    mcep = pysptk.sp2mc(sp, mcep_dim, mcep_alpha)

    return mcep, f0


def _get_basename(path: str) -> str:
    return os.path.splitext(os.path.split(path)[-1])[0]


def _get_best_mcep_params(fs: int) -> Tuple[int, float]:
    if fs == 16000:
        return 23, 0.42
    elif fs == 22050:
        return 34, 0.45
    elif fs == 24000:
        return 34, 0.46
    elif fs == 44100:
        return 39, 0.53
    elif fs == 48000:
        return 39, 0.55
    else:
        raise ValueError(f"Not found the setting for {fs}.")

In [29]:
uids = []
logf0_rmses_recon = []
logf0_corrs_recon = []

for uid in tqdm(val_uids):
    
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(recon_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    eps = 1e-7
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs] + eps)
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs] + eps)

    if len(gen_f0_dtw_voiced) == 0 or len(gt_f0_dtw_voiced) == 0:
        print(f"Skip uid {uid}. len == 0.")
        continue

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")
    uids.append(uid)
    logf0_rmses_recon.append(log_f0_rmse)
    logf0_corrs_recon.append(log_f0_corr)

  0%|          | 0/2772 [00:00<?, ?it/s]

Skip uid 342edcf2-1150-42a0-af79-030571c0996d. len == 0.
Skip uid 2ff974ac-b0f1-4611-8c9f-61c216a8ba96. len == 0.
Skip uid 959dcb2b-4291-45b4-aa24-dc144de6fc1a. len == 0.
Skip uid 94ab3c42-86a9-427c-9dc7-9d3583836f89. len == 0.


D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


Skip uid d7c942a6-bfd3-4ff2-9926-88fafe78781b. len == 0.
Skip uid 9f56d2e9-8536-4ce4-878e-e31f5c64345a. len == 0.
Skip uid 0d885f9d-8786-40d2-ab4b-d6874b5c6507. len == 0.
Skip uid dc0bb287-5310-44a9-b491-b195d98fd53e. len == 0.


In [34]:
v6_logf0_df = pd.DataFrame({
    'uid': uids,
    'logf0_rmse_recon': logf0_rmses_recon,
    'logf0_corr_recon': logf0_corrs_recon,
    # 'logf0_rmse_gt': logf0_rmses_gt,
    # 'logf0_corr_gt': logf0_corrs_gt,
})
v6_logf0_df.to_csv(r".\jupyter_walkthrough\metrics\logF0_0703b_1M2_recon.csv")

ValueError: All arrays must be of the same length

In [31]:
v6_logf0_df = v6_logf0_df.dropna()

In [32]:
print(f"logf0_rmse_recon mean on 0703a_1M2: {v6_logf0_df['logf0_rmse_recon'].values.mean()}")
print(f"logf0_rmse_recon std on 0703a_1M2: {v6_logf0_df['logf0_rmse_recon'].values.std()}")
print(f"logf0_corr_recon mean on 0703a_1M2: {v6_logf0_df['logf0_corr_recon'].values.mean()}")
print(f"logf0_corr_recon std on 0703a_1M2: {v6_logf0_df['logf0_corr_recon'].values.std()}")

logf0_rmse_recon mean on 0703a_1M2: 0.281800061308213
logf0_rmse_recon std on 0703a_1M2: 0.21063260145936688
logf0_corr_recon mean on 0703a_1M2: 0.3964503656060779
logf0_corr_recon std on 0703a_1M2: 0.40082667019104107


In [33]:
uids = []
logf0_rmses_gt = []
logf0_corrs_gt = []

for uid in tqdm(val_uids):
    
    # load wav file as int16
    gen_x, gen_fs = sf.read(op.join(syn_folder, f"{uid}.wav"), dtype="int16")
    gt_x, gt_fs = sf.read(op.join(gt_folder, f"{uid}.wav"), dtype="int16")
    fs = gen_fs
    # extract ground truth and converted features
    gen_mcep, gen_f0 = world_extract(
        x=gen_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    gt_mcep, gt_f0 = world_extract(
        x=gt_x,
        fs=fs,
        f0min=40,
        f0max=800,
        n_fft=1024,
        n_shift=256,
        mcep_dim=None,
        mcep_alpha=None,
    )
    
    # DTW
    _, path = fastdtw(gen_mcep, gt_mcep, dist=spatial.distance.euclidean)
    twf = np.array(path).T
    gen_f0_dtw = gen_f0[twf[0]]
    gt_f0_dtw = gt_f0[twf[1]]
    
    # Get voiced part
    nonzero_idxs = np.where((gen_f0_dtw != 0) & (gt_f0_dtw != 0))[0]
    eps = 1e-7
    gen_f0_dtw_voiced = np.log(gen_f0_dtw[nonzero_idxs] + eps)
    gt_f0_dtw_voiced = np.log(gt_f0_dtw[nonzero_idxs] + eps)

    if len(gen_f0_dtw_voiced) == 0 or len(gt_f0_dtw_voiced) == 0:
        print(f"Skip uid {uid}. len == 0.")
        continue

    # log F0 RMSE
    log_f0_rmse = np.sqrt(np.mean((gen_f0_dtw_voiced - gt_f0_dtw_voiced) ** 2))
    # print(f"{uid} {log_f0_rmse:.4f}")

    # log F0 corr
    log_f0_corr = np.corrcoef(gen_f0_dtw_voiced, gt_f0_dtw_voiced)[0][1]
    # print(f"{uid} {log_f0_corr:.4f}")
    uids.append(uid)
    logf0_rmses_gt.append(log_f0_rmse)
    logf0_corrs_gt.append(log_f0_corr)

  0%|          | 0/2772 [00:00<?, ?it/s]

D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
D:\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Skip uid 2ff974ac-b0f1-4611-8c9f-61c216a8ba96. len == 0.
Skip uid 2ed7e799-e015-4652-bdb1-96b6ecb9c33a. len == 0.
Skip uid 48c252f2-5e53-426d-b275-2974976ce16d. len == 0.
Skip uid b3f48076-f0d8-4209-8bc1-9ac3d0d76e60. len == 0.
Skip uid 76dcc2bc-871e-4ba6-8b8c-5a4cc6a42c4d. len == 0.
Skip uid c40778c5-b585-4897-94ff-6a35edf19add. len == 0.
Skip uid d7c942a6-bfd3-4ff2-9926-88fafe78781b. len == 0.
Skip uid b13f3ab6-fb65-4ae2-9929-4f06d0d58951. len == 0.
Skip uid 934d0029-f16a-4001-9dbc-37924272f3b5. len == 0.
Skip uid 346a8e85-6aff-43d9-87c1-5884a4466686. len == 0.


In [35]:
v6_logf0_df = pd.DataFrame({
    'uid': uids,
    # 'logf0_rmse_recon': logf0_rmses_recon,
    # 'logf0_corr_recon': logf0_corrs_recon,
    'logf0_rmse_gt': logf0_rmses_gt,
    'logf0_corr_gt': logf0_corrs_gt,
})
v6_logf0_df.to_csv(r".\jupyter_walkthrough\metrics\logF0_0703b_1M2_gt.csv")

In [37]:
logf0_rmses_recon_array = np.array(logf0_rmses_recon)[~np.isnan(np.array(logf0_rmses_recon))]
logf0_corrs_recon_array = np.array(logf0_corrs_recon)[~np.isnan(np.array(logf0_corrs_recon))]

print(f"logf0_rmse_recon mean on 0703b_1M2: {logf0_rmses_recon_array.mean()}")
print(f"logf0_rmse_recon std on 0703b_1M2: {logf0_rmses_recon_array.std()}")
print(f"logf0_corr_recon mean on 0703b_1M2: {logf0_corrs_recon_array.mean()}")
print(f"logf0_corr_recon std on 0703b_1M2: {logf0_corrs_recon_array.std()}")

logf0_rmses_gt_array = np.array(logf0_rmses_gt)[~np.isnan(np.array(logf0_rmses_gt))]
logf0_corrs_gt_array = np.array(logf0_corrs_gt)[~np.isnan(np.array(logf0_corrs_gt))]

print(f"logf0_rmse_gt mean on 0703b_1M2: {logf0_rmses_gt_array.mean()}")
print(f"logf0_rmse_gt std on 0703b_1M2: {logf0_rmses_gt_array.std()}")
print(f"logf0_corr_gt mean on 0703b_1M2: {logf0_corrs_gt_array.mean()}")
print(f"logf0_corr_gt std on 0703b_1M2: {logf0_corrs_gt_array.std()}")

logf0_rmse_recon mean on 0703b_1M2: 0.28251933401716595
logf0_rmse_recon std on 0703b_1M2: 0.21249448804620955
logf0_corr_recon mean on 0703b_1M2: 0.3964503656060779
logf0_corr_recon std on 0703b_1M2: 0.40082667019104107
logf0_rmse_gt mean on 0703b_1M2: 0.3133671631279824
logf0_rmse_gt std on 0703b_1M2: 0.23074303607466276
logf0_corr_gt mean on 0703b_1M2: 0.3245314250353599
logf0_corr_gt std on 0703b_1M2: 0.39623624855837947
